## FullAdder 

This notebook walks through the implementation of a full adder circuit.

Start by importing `Magma`.

In [1]:
import magma as m

Define a Python function that constructs a full adder. A full adder has three single bit inputs, and returns the sum and the carry. The sum is the *exclusive or* of the 3 bits, the carry is 1 if any two of the inputs bits are 1. Here is a schematic of a full adder circuit.

![Full Adder](images/full_adder_logisim.png)

The values passed to the `fulladder` function are `Magma` values of type `Bit`.
The Python bitwise operators are overloaded to compute logical functions of the `Magma` values (this corresponds to constructing the circuits to compute these logical functions).

We compute the two desired output values, and return them as a Python tuple. The first element of tuple is the sum, the second element is the carry.

In [2]:
def fulladder(A, B, C):
    return A^B^C, A&B|B&C|C&A

We can test our Python function using native Python values to verify that our implementation behaves as expected. We'll use the standard Python `assert` pattern.

In [3]:
assert fulladder(1, 0, 0) == (1, 0), "Failed"
assert fulladder(0, 1, 0) == (1, 0), "Failed"
assert fulladder(1, 1, 0) == (0, 1), "Failed"
assert fulladder(1, 0, 1) == (0, 1), "Failed"
assert fulladder(1, 1, 1) == (1, 1), "Failed"
print("Success!")

Success!


Now that we have a verified implementation of a `fulladder` in Python, we'll use it construct a `Magma` circuit that is compiled to the IceStick board.

In order to setup everything for the IceStick board,
we first import the class that describes
the board from the module `loam`.
`loam` has definitions for commonly used parts and boards. The definition we're interested in is called `IceStick`.

The IceStick board is based on a Lattice ICE40HX1K FPGA. 
The GPIOs on the FPGA are brought out to two headers named `J1` and `J3`.
The class `IceStick` contains all the parts on the board
and how they are connected via wires.

In [4]:
from loam.boards.icestick import IceStick

import lattice ice40
import lattice mantle40


We start by creating an instance of the IceStick board.

In [5]:
icestick = IceStick()

In these tutorials, 
we adopt the convention that `J1` will be used for inputs
and `J3` will be used for outputs.
We will use these headers to test our full adder by wiring up some switches
and LEDs to the inputs and outputs.

We then configure the first three GPIO pins in `J1` as inputs,
and the first two pins in `J3` as outputs.
We also turn `on` each pin that we are using.
Note the use of *method chaining* to set more than
one option on a GPIO pin.

In [6]:
icestick.J1[0].input().on()
icestick.J1[1].input().on()
icestick.J1[2].input().on()
icestick.J3[0].output().on()
icestick.J3[1].output().on()

J3[1] = GPIO(name="PIO2_16")

With our `icestick` configured, we move on to the setup of the top level `magma` `main` program that runs on the ICE40. 
The arguments to the main program are the headers `J1` and `J3`. We call `fulladder` with three single bit inputs from `J1`. 
We then `wire` the sum and carry outputs returned by `fulladder` to `J3`.

The type of `J1` is is `Bits(3)` and `J3` is `Bits(2)`.
`Bits(n)` is a length `n` array of `Bit` values.
The length of these arrays depend on the number of GPIOs
that have been turned on.
Arrays of bits can be accessed using the standard Python indexing notation (e.g. `[0]`).

We connect the outputs of the full adder to `J3` using a call to `wire`. 
Remember that assigning an object to a Python variable 
creates a named reference to that object
(such as assigning the values returned by `fulladder` to `_sum` and `cout`).
`Magma` values are Python objects,
 so assigning an object to a variable creates a reference to that `Magma` value.

In [7]:
main = icestick.DefineMain()

_sum, cout = fulladder(main.J1[0], main.J1[1], main.J1[2])
m.wire(_sum, main.J3[0])
m.wire(cout, main.J3[1])

When we've finished defining our `main` function, we call the `Magma` function `EndDefine`. Any call to a `Magma` `Define` function such as `DefineMain` or `DefineCircuit` must be accompanied by an `EndDefine` call (similar to how any call to `malloc` must be accompanied by a `free`). This is because `Magma` maintains a stack of definitions, so the `EndDefine` call signals to `Magma` that the current definition on the stack has been completed and should be removed. Failure to call `EndDefine` can lead to nasty error messages that are hard to decipher.

In [8]:
m.EndDefine()

Now we can use the `Magma` `compile` function to generate verilog code targeting a board from the `"lattice"` vendor. The `vendor` parameter tells `Magma` to generate collateral required to program boards from a specific vendor. When the `"lattice"` vendor is specified, `Magma` generates a physical constraints file (pcf) that contains a mapping between physical pin numbers (e.g. 112) and ports on the (compiled) verilog module (e.g. J1[0]).  

In [9]:
m.compile('build/fulladder', main, vendor="lattice")

compiling XOr2
compiling And2
compiling Or2
compiling main


Then we can use `yosys` and the `icestorm` tools to compile and program the FPGA.

In [10]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif fulladder.blif' fulladder.v
arachne-pnr -q -d 1k -o fulladder.txt -p fulladder.pcf fulladder.blif 
icepack fulladder.txt fulladder.bin
iceprog fulladder.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


You can test the program by connecting up some switches and LEDs to the headers.  To see which physical pins correspond to the J3 and J1 pins we used, we can inspect the `pcf` file.

In [11]:
with open("build/fulladder.pcf", "r") as full_adder_pcf:
    print(full_adder_pcf.read())

set_io J1[0] 112
set_io J1[1] 113
set_io J1[2] 114
set_io J3[1] 61
set_io J3[0] 62



Here is an example circuit to see the sum of the inputs displayed on a set of LEDs. We have J1 wired up to the right three switch/LED circuits and J3 wired up to the right two LED circuits (without switches).
![](images/full-adder-seq/IMG_20180610_183451.jpg)
![](images/full-adder-seq/IMG_20180610_183458.jpg)
![](images/full-adder-seq/IMG_20180610_183502.jpg)
![](images/full-adder-seq/IMG_20180610_183505.jpg)
![](images/full-adder-seq/IMG_20180610_183508.jpg)
![](images/full-adder-seq/IMG_20180610_183512.jpg)

## Verilog

If you know verilog, it is instructive to look at the verilog code that was generated.

In [13]:
%cat build/fulladder.v

module XOr2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h6666)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module And2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h8888)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module Or2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'hEEEE)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module main (input [2:0] J1, output [1:0] J3);
wire  inst0_O;
wire  inst1_O;
wire  inst2_O;
wire  inst3_O;
wire  inst4_O;
wire  inst5_O;
wire  inst6_O;
XOr2 inst0 (.I({J1[1],J1[0]}), .O(inst0_O));
XOr2 inst1 (.I({J1[2],inst0_O}), .O(inst1_O));
And2 inst2 (.I({J1[1],J1[0]}), .O(inst2_O));
And2 inst3 (.I({J1[2],J1[1]}), .O(inst3_O));
Or2 inst4 (.I({inst3_O,inst2_O}), .O(inst4_O));
And2 inst5 (.I({J1[0],J1[2]}), .O(inst5_O));
Or2 inst6 (.I({inst5_O,inst4

The logical functions are implemented using verilog modules `And2`, `Or2`, and `XOr2`. These in turn are implemented using 4-bit LUTs using the ICE40 primitive module `SB_LUT4`. The top level `main` module instances the logical functions and wires them up. 
It is all quite simple.